In [1]:
import os
os.environ['SETIGEN_ENABLE_GPU'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from astropy import units as u
import blimpy as bl

import setigen as stg

bshuf filter already loaded, skip it.
zfp filter already loaded, skip it.


Header
Pulsar name        : B0740-28<br>
Frequency Chan0    : 550 MHz<br>
Bandwidth          : 25 MHz<br>
Sideband           : USB<br>
Number of channels : 2048<br>
Sampling Interval  : 81.92 micro-second<br>
Observation date   : 17:03:2021<br>
Observation time   : 20:00:00.544647680<br>
Pol1 raw voltage   : B0740-28_B4_25MHz_P1.rawvlt<br>
Pol2 raw voltage   : B0740-28_B4_25MHz_P2.rawvlt<br>
RAW VLT format     : [[real|imaginary] X NCHAN ] X ntimes<br>
Data length        : 67 Mins<br>

Total intensity    : B0740-28_B4_25MHz_PATI_8int.raw<br>
input params       : gptool.in<br>
rawvlt2power       : user script rawvlt_to_pa_beam.sh (calling rawvlt_to_pa_beam.c)<br>
Profile            : B0740-28.pdf<br>

# Create synthetic data with gaussian noise

In [8]:
sample_rate = 1024e6 #taken as 2*BW
num_taps = 8
num_branches = 1024 

chan_bw = sample_rate / num_branches


In [10]:
chan_bw/1e6

1.0

In [11]:
antenna = stg.voltage.Antenna(sample_rate=sample_rate, 
                              fch1=550*u.MHz,
                              ascending=True,
                              num_pols=1)


In [12]:
antenna.x.sample_rate/1e6

1024.0

In [13]:
antenna.x.add_noise(v_mean=0, 
                    v_std=1)

In [14]:
digitizer = stg.voltage.RealQuantizer(target_fwhm=32,
                                      num_bits=8)

filterbank = stg.voltage.PolyphaseFilterbank(num_taps=num_taps, 
                                             num_branches=num_branches)

requantizer = stg.voltage.ComplexQuantizer(target_fwhm=32,
                                           num_bits=8)



rvb = stg.voltage.RawVoltageBackend(antenna,
                                    digitizer=digitizer,
                                    filterbank=filterbank,
                                    requantizer=requantizer,
                                    start_chan=0,
                                    num_chans=64,
                                    block_size=134217728,
                                    blocks_per_file=128,
                                    num_subblocks=256)


    




return int(obs_length * abs(self.chan_bw) * self.num_antennas * self.num_chans * self.bytes_per_sample / self.block_size)

In [15]:
rvb.get_num_blocks(20)

19

In [20]:
print(antenna.sample_rate)
print(rvb.sample_rate/1e6)

print(rvb.num_branches)
print(rvb.chan_bw)
print(rvb.tbin)
print(rvb.fch1/1e6)
print('final channel freq=',(rvb.fch1+chan_bw*rvb.num_chans)/1e6)

1024000000.0
1024.0
1024
1000000.0
1e-06
550.0
final channel freq= 614.0


614

In [23]:
rvb.record(output_file_stem='/home/eakshay/setigen/data/guppi_with_chanbw_1mhz',
           num_blocks=2, 
           length_mode='num_blocks',
           header_dict={'TELESCOP': 'GBT'},
           verbose=False)


Blocks: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.80s/it]


In [ ]:
#!rawspec -f 1024 -t 4 -d . example_snr

In [24]:
start_chan = 0
input_file_stem = '/home/eakshay/setigen/data/guppi_with_chanbw_1mhz'

raw_params = stg.voltage.get_raw_params(input_file_stem=input_file_stem,
                                        start_chan=start_chan)

In [25]:
raw_params

{'num_bits': 8,
 'chan_bw': 1000000.0,
 'ascending': True,
 'num_pols': 1,
 'block_size': 134217728,
 'obs_length': 2.097152,
 'tbin': 1e-06,
 'num_antennas': 1,
 'num_chans': 64,
 'center_freq': 581500000.0,
 'fch1': 550000000.0}

In [59]:
551538085.9375/1e6

551.5380859375